- See /docs/README.md

In [37]:
import json

v4_filename = "Tristesse d'Olympio.corpus.json"
with open(v4_filename, "rt", encoding="utf-8") as fin:
	v4 = json.load(fin)

ps = v4["paragraphs"]
for p in ps[:2]:
	print(p)

{'pstate': 'ANNOTATED', 'tokens': [{'txt': 'Les', 'is_delimiter': False, 'gloss': 'The'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'champs', 'is_delimiter': False, 'gloss': 'fields'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': "n'étaient", 'is_delimiter': False, 'gloss': "weren't"}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'point', 'is_delimiter': False, 'gloss': 'quite'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'noirs,', 'is_delimiter': False, 'gloss': 'black,'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'les', 'is_delimiter': False, 'gloss': 'the'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'cieux', 'is_delimiter': False, 'gloss': 'heavens'}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': "n'étaient", 'is_delimiter': False, 'gloss': "weren't"}, {'txt': ' ', 'is_delimiter': True, 'gloss': None}, {'txt': 'pas', 'is_delimiter': False, 'gloss': 'not'}, {'txt': ' ', 'is_del

In [38]:
# Naive
import uuid

title = v4_filename.replace(".corpus.json", "")
author = "Victor Hugo"
annotation_info = "Machine-glossed with fine-tuned gpt-4o-mini (https://github.com/parkchamchi/GlossySnake/blob/master/src/tools/data/gs_240918.jsonl)"
original_language = "fr"
gloss_language = "en"
note = "Initially generated as a v4 file."
unique_identifier = uuid.uuid4().int

p_out_s = []
gs_out_s = []
for p_id, p in enumerate(v4["paragraphs"]):
	p_out = ""
	g_out_s = []
	for t_id, t in enumerate(p["tokens"]):
		t_out = t["txt"]
		is_delimiter = t["is_delimiter"]
		if not is_delimiter:
			t_out = f"<token id='{ t_id }'>{ t_out }</token>" 

			g_out = t["gloss"]
			special = ""
			if g_out == "!UNKNOWN":
				special = " special='unknown'"
			elif g_out == "!TO_REANNOTATE":
				special = " special='to-reannotate'"
			g_out = f"<gloss-for-token for='{ t_id }'{ special }>{ g_out }</gloss-for-token>"
			g_out_s.append(g_out)
		p_out += t_out

	p_out = f"<paragraph id='{ p_id }'>{ p_out }</paragraph>"
	p_out_s.append(p_out)

	if g_out_s != []:
		gs_out = ''.join(g_out_s)
		gs_out = f"<glosses-for-paragraph for='{ p_id }'>{ gs_out }</glosses-for-paragraph>"
		gs_out_s.append(gs_out)
	
newline = '\n\t'
out = f"""<?xml version="1.0" encoding="UTF-8" ?>
<glossysnake-text version="5">
	<metadata>
		<item type="title">{ title }</item>
		<item type="author">{ author }</item>
		<item type="annotation-info">{ annotation_info }</item>
		<item type="original-language">{ original_language }</item>
		<item type="gloss-language">{ gloss_language }</item>
		<item type="note">{ note }</item>
		<item type="unique-identifier">{ unique_identifier }</item>
	</metadata>

	<paragraphs>{ ''.join(p_out_s) }</paragraphs>

	<glosses>
		{ newline.join(gs_out_s) }
	</glosses>
</glossysnake-text>"""

In [39]:
with open(v4_filename.replace(".json", ".xml"), "wt", encoding="utf-8") as fout:
	fout.write(out)